# 影片轉df函數

In [1]:
import cv2
import mediapipe as mp
import pandas as pd

def generate_pose_data(input_path):
    # 初始化 MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=2,
        enable_segmentation=False,
        min_detection_confidence=0.5
    )
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    # 輸入影片路徑
    video_path = input_path
    cap = cv2.VideoCapture(video_path)

    # 儲存結果
    results = []

    frame_idx = 0
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # 轉換顏色為 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results_pose = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 取得姿勢關鍵點並畫在畫面上
        if results_pose.pose_landmarks:
            landmarks = results_pose.pose_landmarks.landmark
            for i, lm in enumerate(landmarks):
                results.append({
                    "frame": frame_idx,
                    "joint_id": i,
                    "x": lm.x,
                    "y": lm.y,
                    "z": lm.z,
                    "visibility": lm.visibility
                })

            # 繪製骨架
            mp_drawing.draw_landmarks(
                image,
                results_pose.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

        frame_idx += 1

        # 顯示畫面
        cv2.imshow("Pose Skeleton Overlay", image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pose.close()

    # 儲存為 CSV
    df = pd.DataFrame(results)
    return df 

# 依序將每一個影片轉成df格式

In [2]:
import os
import glob
from tqdm import tqdm

df_list = {}

# 假設 raw_video 是資料夾路徑
folder_path = "../clip_video"

# 利用 glob 找出所有符合 pitch_*.mp4 的檔案
file_paths = glob.glob(os.path.join(folder_path, "pitch_*.mp4"))

for path in tqdm(file_paths):
    # 取出標號，假設是 pitch_後面接數字，檔名格式固定
    filename = os.path.basename(path)  # 取檔名
    # 取 pitch_ 後面跟著的標號 (不含副檔名)
    # e.g. pitch_1.mp4 -> 1
    pitch_id = filename.replace("pitch_", "").replace(".mp4", "")
    
    # 呼叫 generate_pose_data 並存到字典
    df_list[pitch_id] = generate_pose_data(path)


100%|██████████| 10/10 [03:59<00:00, 23.98s/it]


In [3]:
df_list.keys()

dict_keys(['0001_cropped_2_4', '0002_cropped_2_4', '0003_cropped_2_4', '0004_cropped_2_4', '0005_cropped_2_4', '0006_cropped_2_4', '0007_cropped_2_4', '0008_cropped_2_4', '0009_cropped_2_4', '0010_cropped_2_4'])

In [4]:
df_list[df_list.keys()[0]]

TypeError: 'dict_keys' object is not subscriptable

In [ ]:
import numpy as np
all_data = []
for i in tqdm(df_list.keys()):
    # 假設這是你的資料
    df = df_list[i]

    # 使用 pivot 將 joint_id 對應的 x, y, z 展開成欄位
    df_pivot = df.pivot(index='frame', columns='joint_id', values=['x', 'y', 'z'])

    # 將 multi-index 欄位轉為單一層級，欄位名稱格式為 'x_0', 'y_0', ..., 'z_32'
    df_pivot.columns = [f'{coord}_{jid}' for coord, jid in df_pivot.columns]

    # 如果需要重設 index（例如 frame 從欄位轉為普通欄）
    df_pivot = df_pivot.reset_index(drop=True)

    # 顯示結果
    all_data.append(df_pivot.values[:100])
all_data = np.array(all_data)

100%|██████████| 10/10 [00:00<00:00, 425.32it/s]


In [ ]:
all_data.shape
#保存all_data到train_data資料夾 命名為all_data #all_data是一個np array
import numpy as np
import os

# 確保資料夾存在
os.makedirs('../train_data', exist_ok=True)

# 儲存 all_data 為 train_data/all_data.npy
np.save('../train_data/all_data.npy', all_data)
